In [3]:
# import Pkg;
# Pkg.add("HiGHS")
# Pkg.add("JuMP")

### Question 1: Written Model

**Variable defintions**: \
x<sub>1</sub>: Raw Carrots \
x<sub>2</sub>: Baked Potatoes \
x<sub>3</sub>: Wheat Bread \
x<sub>4</sub>: Cheddar Cheese \
x<sub>5</sub>: Peanut Butter

**Objective Function**: \
min 0.14x<sub>1</sub> + 0.12x<sub>2</sub> + 0.2x<sub>3</sub> + 0.75x<sub>4</sub> + 0.15x<sub>5</sub>

**Constraints**: \
23x<sub>1</sub> + 171x<sub>2</sub> + 65x<sub>3</sub> + 112x<sub>4</sub> + 188x<sub>5</sub> >= 2000 \
0.1x<sub>1</sub> + 0.2x<sub>2</sub> + 9.3x<sub>4</sub> + 16x<sub>5</sub> >= 50 \
0.6x<sub>1</sub> + 3.7x<sub>2</sub> + 2.2x<sub>3</sub> + 7x<sub>4</sub> + 7.7x<sub>5</sub> >= 100 \
6x<sub>1</sub> + 30x<sub>2</sub> + 13x<sub>3</sub> + 2x<sub>5</sub> >= 250

In [1]:
using HiGHS;
using JuMP;

In [15]:
toddler = Model(HiGHS.Optimizer)

@variable(toddler, x1>=0);
@variable(toddler, x2>=0);
@variable(toddler, x3>=0);
@variable(toddler, x4>=0);
@variable(toddler, x5>=0);

@constraint(toddler, calories, 23x1+171x2+65x3+112x4+188x5 >= 2000);
@constraint(toddler, fat, 0.1x1+0.2x2+9.3x4+16x5 >= 50);
@constraint(toddler, protein, 0.6x1+3.7x2+2.2x3+7x4+7.7x5 >= 100);
@constraint(toddler, carbs, 6x1+30x2+13x3+2x5 >= 250);

@objective(toddler, Min, 0.14x1+0.12x2+0.2x3+0.75x4+0.15x5);

In [9]:
optimize!(toddler);

Coefficient ranges:
  Matrix [1e-01, 2e+02]
  Cost   [1e-01, 8e-01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+01, 2e+03]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  2.3177549195e+00
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [10]:
@show objective_value(toddler)
@show value(x1)
@show value(x2)
@show value(x3)
@show value(x4)
@show value(x5)

objective_value(toddler) = 2.3177549194991056
value(x1) = 0.0
value(x2) = 7.714669051878355
value(x3) = 0.0
value(x4) = 0.0
value(x5) = 9.279964221824686


9.279964221824686

### Question 2: Written Model

In [2]:
types = [:colombian, :kenyan, :indonesian]
blends = [:easydoesit, :slowdown]

supply = Dict(zip(types, [550, 450, 650]))
cost = Dict(zip(types, [1, 0.85, 1.55]))
revenue = Dict(zip(blends, [3.95, 4.25]))

Dict{Symbol, Float64} with 2 entries:
  :slowdown   => 4.25
  :easydoesit => 3.95

In [18]:
coffee = Model(HiGHS.Optimizer)

@variable(coffee, x[types, blends] >= 0)

@expression(coffee, easy_total, sum(x[i, :easydoesit] for i in types))
@expression(coffee, slow_total, sum(x[i, :slowdown] for i in types))

x[colombian,slowdown] + x[kenyan,slowdown] + x[indonesian,slowdown]

In [19]:
# Easydoesit Constraint
@constraint(coffee, sum(x[:kenyan, :easydoesit]) >= 0.50 * easy_total)
@constraint(coffee, sum(x[:kenyan, :easydoesit]) <= 0.60 * easy_total)    
@constraint(coffee, sum(x[:indonesian, :easydoesit]) <= 0.20 * easy_total)

-0.2 x[colombian,easydoesit] - 0.2 x[kenyan,easydoesit] + 0.8 x[indonesian,easydoesit] <= 0

In [20]:
# Slowdown Constraint
@constraint(coffee, sum(x[:indonesian, :slowdown]) >= 0.60 * slow_total)
@constraint(coffee, sum(x[:indonesian, :slowdown]) <= 0.75 * slow_total)
@constraint(coffee, sum(x[:colombian, :slowdown]) >= 0.10 * slow_total)  

0.9 x[colombian,slowdown] - 0.1 x[kenyan,slowdown] - 0.1 x[indonesian,slowdown] >= 0

In [21]:
# Supply Constraint
@constraint(coffee, [i in types], sum(x[i, j] for j in blends) <= supply[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [:colombian, :kenyan, :indonesian]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[colombian,easydoesit] + x[colombian,slowdown] <= 550
 x[kenyan,easydoesit] + x[kenyan,slowdown] <= 450
 x[indonesian,easydoesit] + x[indonesian,slowdown] <= 650

In [22]:
# Objective Function
@objective(coffee, Max, sum(revenue[j] * sum(x[i, j] for i in types) for j in blends) - sum(cost[i] * sum(x[i, j] for j in blends) for i in types))
print(coffee)

Max 2.95 x[colombian,easydoesit] + 3.1 x[kenyan,easydoesit] + 2.4000000000000004 x[indonesian,easydoesit] + 3.25 x[colombian,slowdown] + 3.4 x[kenyan,slowdown] + 2.7 x[indonesian,slowdown]
Subject to
 -0.5 x[colombian,easydoesit] + 0.5 x[kenyan,easydoesit] - 0.5 x[indonesian,easydoesit] >= 0
 -0.6 x[colombian,slowdown] - 0.6 x[kenyan,slowdown] + 0.4 x[indonesian,slowdown] >= 0
 0.9 x[colombian,slowdown] - 0.1 x[kenyan,slowdown] - 0.1 x[indonesian,slowdown] >= 0
 -0.6 x[colombian,easydoesit] + 0.4 x[kenyan,easydoesit] - 0.6 x[indonesian,easydoesit] <= 0
 -0.2 x[colombian,easydoesit] - 0.2 x[kenyan,easydoesit] + 0.8 x[indonesian,easydoesit] <= 0
 -0.75 x[colombian,slowdown] - 0.75 x[kenyan,slowdown] + 0.25 x[indonesian,slowdown] <= 0
 x[colombian,easydoesit] + x[colombian,slowdown] <= 550
 x[kenyan,easydoesit] + x[kenyan,slowdown] <= 450
 x[indonesian,easydoesit] + x[indonesian,slowdown] <= 650
 x[colombian,easydoesit] >= 0
 x[kenyan,easydoesit] >= 0
 x[indonesian,easydoesit] >= 0
 x[col

In [42]:
# Solve
optimize!(coffee)
println()
for i in blends
    println("$i: ", value(sum(x[j, i] for j in types)))
end
println()
for i in blends
    for j in types
        println("$i $j: ", value(x[j, i])) 
    end
end

println("\nOptimal Profit: ", objective_value(coffee))

Coefficient ranges:
  Matrix [1e-01, 1e+00]
  Cost   [2e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 6e+02]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  4.9025000000e+03
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

easydoesit: 566.6666666666665
slowdown: 1083.3333333333335

easydoesit colombian: 283.33333333333354
easydoesit kenyan: 283.333333333333
easydoesit indonesian: 0.0
slowdown colombian: 266.66666666666646
slowdown kenyan: 166.6666666666669
slowdown indonesian: 650.0000000000002

Optimal Profit: 4902.5
